In [1]:
!pip -q install mlxtend

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [2]:
ADULTS_PATH = "consumption_user_adulti.csv"
CHILDREN_PATH = "consumption_user_copii.csv"

adults_raw = pd.read_csv(ADULTS_PATH, sep=",", engine="python", on_bad_lines="skip")
children_raw = pd.read_csv(CHILDREN_PATH, sep=",", engine="python", on_bad_lines="skip")

print("Adults shape:", adults_raw.shape)
print("Children shape:", children_raw.shape)

print("\nAdults columns sample:", list(adults_raw.columns)[:15])
print("Children columns sample:", list(children_raw.columns)[:15])


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Adults shape: (257952, 80)
Children shape: (20851, 80)

Adults columns sample: ['SUBJECT', 'ROUND', 'SURVEY_DAY', 'RESPONDER', 'SEASON', 'CONSUMPTION_DAY', 'CONSUMPTION_MONTH', 'CONSUMPTION_YEAR', 'WEEK_DAY', 'EXCEPTION_DAY', 'CONSUMPTION_TIME_HOUR', 'CONSUMPTION_TIME_MINUTES', 'MEAL_NAME', 'PLACE', 'FOOD_TYPE']
Children columns sample: ['SUBJECT', 'ROUND', 'SURVEY_DAY', 'RESPONDER', 'SEASON', 'CONSUMPTION_DAY', 'CONSUMPTION_MONTH', 'CONSUMPTION_YEAR', 'WEEK_DAY', 'EXCEPTION_DAY', 'CONSUMPTION_TIME_HOUR', 'CONSUMPTION_TIME_MINUTES', 'MEAL_NAME', 'PLACE', 'FOOD_TYPE']


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [4]:
# coloanele esențiale pentru reguli de asociere
COLS = [
    "SUBJECT",
    "SURVEY_DAY",
    "FOODEX2_INGR_DESCR",
    "FOOD_AMOUNT_CONS",
    "FOOD_AMOUNT_REPORTED"
]

adults = adults_raw[[c for c in COLS if c in adults_raw.columns]].copy()
children = children_raw[[c for c in COLS if c in children_raw.columns]].copy()

print("Adults cleaned:", adults.shape)
print("Children cleaned:", children.shape)


Adults cleaned: (257952, 5)
Children cleaned: (20851, 5)


In [5]:
def build_transactions(df):
    df = df.dropna(subset=["SUBJECT", "SURVEY_DAY", "FOODEX2_INGR_DESCR"]).copy()

    # alegem coloana de cantitate disponibilă
    qty_col = "FOOD_AMOUNT_CONS" if "FOOD_AMOUNT_CONS" in df.columns else "FOOD_AMOUNT_REPORTED"
    if qty_col in df.columns:
        df = df[df[qty_col].fillna(0) > 0]

    df["item"] = df["FOODEX2_INGR_DESCR"].astype(str).str.strip()

    # tranzacție = (SUBJECT, SURVEY_DAY)
    transactions = (
        df.groupby(["SUBJECT", "SURVEY_DAY"])["item"]
          .apply(lambda x: sorted(set(x)))
          .tolist()
    )
    return transactions

trans_A = build_transactions(adults)
trans_C = build_transactions(children)

print("Adults transactions:", len(trans_A))
print("Children transactions:", len(trans_C))
print("Example Adults transaction:", trans_A[0][:10])


Adults transactions: 9666
Children transactions: 773
Example Adults transaction: ['Beans (dry) and similar-, RACSOURCE= Beans (fresh seeds without pods) and similar-, PROCESS= Cooking in water', 'Black pepper, PROCESS= Stewing', 'Butter', 'Butter, PROCESS= Raw, no heat treatment', 'Carrots, PROCESS= Cooking in water', 'Celeries, PROCESS= Cooking in water', 'Chicken liver, PROCESS= Pan frying / shallow frying', 'Coffee (average strength) beverage, PROCESS= Boiling', 'Fresh raw sausages, PROCESS= Pan frying / shallow frying', 'Gherkins, PROCESS= Pickling']


In [6]:
from mlxtend.preprocessing import TransactionEncoder

def make_basket(transactions):
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    basket = pd.DataFrame(te_ary, columns=te.columns_)
    return basket

basket_A = make_basket(trans_A)
basket_C = make_basket(trans_C)

print("Basket Adults:", basket_A.shape)
print("Basket Children:", basket_C.shape)

print("\nTop 10 items Adults:")
print(basket_A.sum().sort_values(ascending=False).head(10))

print("\nTop 10 items Children:")
print(basket_C.sum().sort_values(ascending=False).head(10))


Basket Adults: (9666, 1558)
Basket Children: (773, 849)

Top 10 items Adults:
Wheat bread and rolls, white (refined flour)                                          7640
Salt, PROCESS= Raw, no heat treatment                                                 4703
Coffee (average strength) beverage, PROCESS= Boiling                                  4487
Tap water, PROCESS= Boiling                                                           3929
Sunflower seed oil, edible, PROCESS= Raw, no heat treatment                           3473
Sunflower seed oil, edible, PROCESS= Stewing, PROCESS= Pan frying / shallow frying    3209
Onions, PROCESS= Stewing, PROCESS= Pan frying / shallow frying                        3141
Vegetables and vegetable products, GEN= Other, PROCESS= Raw, no heat treatment        2634
Carrots, PROCESS= Stewing, PROCESS= Pan frying / shallow frying                       2467
Tap water, PROCESS= Poaching                                                          2427
dtype: int64

In [7]:
def topN_basket(basket, topN=300):
    keep = basket.sum().sort_values(ascending=False).head(topN).index
    return basket[keep]

basket_A_small = topN_basket(basket_A, topN=300)
basket_C_small = topN_basket(basket_C, topN=300)

def mine_rules_fast(basket, min_support=0.03, min_confidence=0.35, min_lift=1.1, max_len=2):
    freq = fpgrowth(basket, min_support=min_support, use_colnames=True, max_len=max_len)
    rules = association_rules(freq, metric="confidence", min_threshold=min_confidence)
    rules = rules[rules["lift"] >= min_lift].copy()
    rules = rules.sort_values(["lift", "confidence", "support"], ascending=False)
    return freq, rules

freq_A, rules_A = mine_rules_fast(basket_A_small)
freq_C, rules_C = mine_rules_fast(basket_C_small)

print("Adults:", freq_A.shape, rules_A.shape)
print("Children:", freq_C.shape, rules_C.shape)

rules_A.head(10)[["antecedents","consequents","support","confidence","lift"]]


Adults: (1858, 2) (1138, 14)
Children: (2049, 2) (1259, 14)


,antecedents,consequents,support,confidence,lift
1554,"(Mammals and birds meat, PROCESS= Pan frying /...","(Garlic, PROCESS= Pan frying / shallow frying)",0.033106,1.000000,28.597633
1553,"(Garlic, PROCESS= Pan frying / shallow frying)","(Mammals and birds meat, PROCESS= Pan frying /...",0.033106,0.946746,28.597633
1555,"(Mammals and birds meat, PROCESS= Pan frying /...","(Wheat bread and rolls, white (refined flour),...",0.033106,1.000000,27.617143
1556,"(Wheat bread and rolls, white (refined flour),...","(Mammals and birds meat, PROCESS= Pan frying /...",0.033106,0.914286,27.617143
1549,"(Wheat bread and rolls, white (refined flour),...","(Garlic, PROCESS= Pan frying / shallow frying)",0.034037,0.940000,26.881775
1548,"(Garlic, PROCESS= Pan frying / shallow frying)","(Wheat bread and rolls, white (refined flour),...",0.034037,0.973373,26.881775
1303,"(Mammals and birds meat, PROCESS= Cooking in w...","(Juice, tomato, PROCESS= Cooking in water)",0.042624,1.000000,23.461165
1304,"(Juice, tomato, PROCESS= Cooking in water)","(Mammals and birds meat, PROCESS= Cooking in w...",0.042624,1.000000,23.461165
1558,"(Mammals and birds meat, PROCESS= Pan frying /...","(Onions, PROCESS= Pan frying / shallow frying)",0.033106,1.000000,23.461165
1557,"(Onions, PROCESS= Pan frying / shallow frying)","(Mammals and birds meat, PROCESS= Pan frying /...",0.033106,0.776699,23.461165


In [8]:
basket_C_small = topN_basket(basket_C, topN=600)

freq_C, rules_C = mine_rules_fast(
    basket_C_small,
    min_support=0.03,
    min_confidence=0.35,
    min_lift=1.1,
    max_len=2
)

print("Children:", freq_C.shape, rules_C.shape)
rules_C.head(10)[["antecedents","consequents","support","confidence","lift"]]


Children: (2049, 2) (1259, 14)


,antecedents,consequents,support,confidence,lift
873,"(Cocoa powder, PROCESS= Reheating)","(White sugar, PROCESS= Reheating)",0.038810,1.000000,25.766667
874,"(White sugar, PROCESS= Reheating)","(Cocoa powder, PROCESS= Reheating)",0.038810,1.000000,25.766667
876,"(Milk, PROCESS= Reheating)","(White sugar, PROCESS= Reheating)",0.038810,1.000000,25.766667
877,"(White sugar, PROCESS= Reheating)","(Milk, PROCESS= Reheating)",0.038810,1.000000,25.766667
878,"(Milk, PROCESS= Reheating)","(Cocoa powder, PROCESS= Reheating)",0.038810,1.000000,25.766667
879,"(Cocoa powder, PROCESS= Reheating)","(Milk, PROCESS= Reheating)",0.038810,1.000000,25.766667
1445,"(Milk, PROCESS= Stewing)","(Rice grain, PROCESS= Stewing)",0.043984,1.000000,22.085714
1446,"(Rice grain, PROCESS= Stewing)","(Milk, PROCESS= Stewing)",0.043984,0.971429,22.085714
995,"(Cocoa powder, PROCESS= Boiling)","(White sugar, PROCESS= Raw, no heat treatment)",0.034929,0.964286,20.145753
994,"(White sugar, PROCESS= Raw, no heat treatment)","(Cocoa powder, PROCESS= Boiling)",0.034929,0.729730,20.145753


In [9]:
SUBJ_ADULTS_PATH = "subject_user_adulti.csv"
SUBJ_CHILDREN_PATH = "subject_user_copii.csv"

subj_adults = pd.read_csv(SUBJ_ADULTS_PATH, sep=",", engine="python", on_bad_lines="skip")
subj_children = pd.read_csv(SUBJ_CHILDREN_PATH, sep=",", engine="python", on_bad_lines="skip")

print("Subjects Adults:", subj_adults.shape)
print("Subjects Children:", subj_children.shape)

print("\nAdults subject columns:", list(subj_adults.columns))
print("\nChildren subject columns:", list(subj_children.columns))


Subjects Adults: (1382, 23)
Subjects Children: (773, 23)

Adults subject columns: ['ADM0_NAME', 'ADM1_NAME', 'ADM2_NAME', 'WEIGHTING_FACTOR', 'AREA_TYPE', 'HOUSEHOLD', 'SUBJECT', 'ROUND', 'SEX', 'AGE_YEAR', 'ANT_DAY', 'ANT_MONTH', 'ANT_YEAR', 'WEIGHT', 'HEIGHT', 'METHOD_WEIGHT', 'METHOD_HEIGHT', 'SPECIAL_DIET', 'PREG_LACT', 'BREASTFEEDING', 'UNOVERREP', 'ACTIVITY', 'VERSION']

Children subject columns: ['ADM0_NAME', 'ADM1_NAME', 'ADM2_NAME', 'WEIGHTING_FACTOR', 'AREA_TYPE', 'HOUSEHOLD', 'SUBJECT', 'ROUND', 'SEX', 'AGE_YEAR', 'ANT_DAY', 'ANT_MONTH', 'ANT_YEAR', 'WEIGHT', 'HEIGHT', 'METHOD_WEIGHT', 'METHOD_HEIGHT', 'SPECIAL_DIET', 'PREG_LACT', 'BREASTFEEDING', 'UNOVERREP', 'ACTIVITY', 'VERSION']


In [10]:
# păstrăm doar SUBJECT + SEX
subj_adults_small = subj_adults[["SUBJECT", "SEX"]].copy()
subj_children_small = subj_children[["SUBJECT", "SEX"]].copy()

print(subj_adults_small.head())
print(subj_children_small.head())

print("\nAdults SEX values:", subj_adults_small["SEX"].value_counts())
print("Children SEX values:", subj_children_small["SEX"].value_counts())


   SUBJECT  SEX
0        1    1
1        2    1
2        3    1
3        4    1
4        5    2
   SUBJECT  SEX
0        1    1
1        2    2
2        3    1
3        4    2
4        5    1

Adults SEX values: SEX
2    743
1    639
Name: count, dtype: int64
Children SEX values: SEX
1    402
2    371
Name: count, dtype: int64


In [11]:
# JOIN adults
adults_sex = adults.merge(
    subj_adults_small,
    on="SUBJECT",
    how="inner"
)

# JOIN children
children_sex = children.merge(
    subj_children_small,
    on="SUBJECT",
    how="inner"
)

print("Adults after join:", adults_sex.shape)
print("Children after join:", children_sex.shape)

print("\nAdults SEX distribution:")
print(adults_sex["SEX"].value_counts())

print("\nChildren SEX distribution:")
print(children_sex["SEX"].value_counts())


Adults after join: (257952, 6)
Children after join: (20851, 6)

Adults SEX distribution:
SEX
2    134548
1    123404
Name: count, dtype: int64

Children SEX distribution:
SEX
1    11083
2     9768
Name: count, dtype: int64


In [12]:
def build_transactions_by_sex(df, sex_value):
    df_s = df[df["SEX"] == sex_value].copy()
    df_s = df_s.dropna(subset=["SUBJECT", "SURVEY_DAY", "FOODEX2_INGR_DESCR"])

    qty_col = "FOOD_AMOUNT_CONS" if "FOOD_AMOUNT_CONS" in df_s.columns else "FOOD_AMOUNT_REPORTED"
    if qty_col in df_s.columns:
        df_s = df_s[df_s[qty_col].fillna(0) > 0]

    df_s["item"] = df_s["FOODEX2_INGR_DESCR"].astype(str).str.strip()

    transactions = (
        df_s.groupby(["SUBJECT", "SURVEY_DAY"])["item"]
            .apply(lambda x: sorted(set(x)))
            .tolist()
    )
    return transactions


In [13]:
# Adults
trans_A_m = build_transactions_by_sex(adults_sex, 1)  # male
trans_A_f = build_transactions_by_sex(adults_sex, 2)  # female

# Children
trans_C_b = build_transactions_by_sex(children_sex, 1)  # boys
trans_C_g = build_transactions_by_sex(children_sex, 2)  # girls

print("Adults male tx:", len(trans_A_m))
print("Adults female tx:", len(trans_A_f))
print("Children boys tx:", len(trans_C_b))
print("Children girls tx:", len(trans_C_g))


Adults male tx: 4469
Adults female tx: 5197
Children boys tx: 402
Children girls tx: 371


In [14]:
# 1) basket
basket_A_m = make_basket(trans_A_m)
basket_A_f = make_basket(trans_A_f)
basket_C_b = make_basket(trans_C_b)
basket_C_g = make_basket(trans_C_g)

# 2) reducem itemii (diferențiat, ca să păstrăm specificul)
basket_A_m_small = topN_basket(basket_A_m, topN=300)
basket_A_f_small = topN_basket(basket_A_f, topN=300)
basket_C_b_small = topN_basket(basket_C_b, topN=500)
basket_C_g_small = topN_basket(basket_C_g, topN=500)

# 3) FP-Growth + reguli
freq_A_m, rules_A_m = mine_rules_fast(basket_A_m_small)
freq_A_f, rules_A_f = mine_rules_fast(basket_A_f_small)
freq_C_b, rules_C_b = mine_rules_fast(basket_C_b_small)
freq_C_g, rules_C_g = mine_rules_fast(basket_C_g_small)

print("Adults male:", freq_A_m.shape, rules_A_m.shape)
print("Adults female:", freq_A_f.shape, rules_A_f.shape)
print("Children boys:", freq_C_b.shape, rules_C_b.shape)
print("Children girls:", freq_C_g.shape, rules_C_g.shape)


Adults male: (2008, 2) (1216, 14)
Adults female: (1810, 2) (1137, 14)
Children boys: (2238, 2) (1324, 14)
Children girls: (1854, 2) (1311, 14)


In [15]:
def top_items_from_basket(basket, topN=10):
    return (
        basket.sum()
              .sort_values(ascending=False)
              .head(topN)
              .reset_index()
              .rename(columns={"index": "Food", 0: "Frequency"})
    )

# Adults
top_A_m = top_items_from_basket(basket_A_m)
top_A_f = top_items_from_basket(basket_A_f)

# Children
top_C_b = top_items_from_basket(basket_C_b)
top_C_g = top_items_from_basket(basket_C_g)

print("=== Adults Male ===")
display(top_A_m)

print("=== Adults Female ===")
display(top_A_f)

print("=== Children Boys ===")
display(top_C_b)

print("=== Children Girls ===")
display(top_C_g)


=== Adults Male ===


,Food,Frequency
0,"Wheat bread and rolls, white (refined flour)",3804
1,"Salt, PROCESS= Raw, no heat treatment",2205
2,"Coffee (average strength) beverage, PROCESS= B...",1984
3,"Tap water, PROCESS= Boiling",1884
4,"Sunflower seed oil, edible, PROCESS= Raw, no h...",1620
5,"Sunflower seed oil, edible, PROCESS= Stewing, ...",1533
6,"Onions, PROCESS= Stewing, PROCESS= Pan frying ...",1501
7,"Carrots, PROCESS= Stewing, PROCESS= Pan frying...",1197
8,"Vegetables and vegetable products, GEN= Other,...",1180
9,"Tap water, PROCESS= Poaching",1157


=== Adults Female ===


,Food,Frequency
0,"Wheat bread and rolls, white (refined flour)",3836
1,"Coffee (average strength) beverage, PROCESS= B...",2503
2,"Salt, PROCESS= Raw, no heat treatment",2498
3,"Tap water, PROCESS= Boiling",2045
4,"Sunflower seed oil, edible, PROCESS= Raw, no h...",1853
5,"Sunflower seed oil, edible, PROCESS= Stewing, ...",1676
6,"Onions, PROCESS= Stewing, PROCESS= Pan frying ...",1640
7,"Vegetables and vegetable products, GEN= Other,...",1454
8,"Carrots, PROCESS= Stewing, PROCESS= Pan frying...",1270
9,"Tap water, PROCESS= Poaching",1270


=== Children Boys ===


,Food,Frequency
0,"Wheat bread and rolls, white (refined flour)",342
1,"Salt, PROCESS= Raw, no heat treatment",197
2,"Tap water, PROCESS= Boiling",171
3,"Sunflower seed oil, edible, PROCESS= Stewing, ...",155
4,"Onions, PROCESS= Stewing, PROCESS= Pan frying ...",150
5,Milk,141
6,"Sunflower seed oil, edible, PROCESS= Raw, no h...",128
7,"Carrots, PROCESS= Stewing, PROCESS= Pan frying...",120
8,"Salt, PROCESS= Stewing, PROCESS= Pan frying / ...",112
9,"Herbal and other non-tea infusions, PROCESS= B...",110


=== Children Girls ===


,Food,Frequency
0,"Wheat bread and rolls, white (refined flour)",290
1,"Salt, PROCESS= Raw, no heat treatment",165
2,"Tap water, PROCESS= Boiling",157
3,"Sunflower seed oil, edible, PROCESS= Stewing, ...",126
4,"Sunflower seed oil, edible, PROCESS= Raw, no h...",124
5,"Onions, PROCESS= Stewing, PROCESS= Pan frying ...",123
6,Milk,118
7,Apples,104
8,"Carrots, PROCESS= Stewing, PROCESS= Pan frying...",98
9,"Herbal and other non-tea infusions, PROCESS= B...",98


In [16]:
def top_items_percent(basket, topN=10):
    pct = (basket.sum() / basket.shape[0]) * 100
    return (
        pct.sort_values(ascending=False)
           .head(topN)
           .reset_index()
           .rename(columns={"index": "Food", 0: "Percent_of_transactions"})
    )

top_A_m_pct = top_items_percent(basket_A_m)
top_A_f_pct = top_items_percent(basket_A_f)
top_C_b_pct = top_items_percent(basket_C_b)
top_C_g_pct = top_items_percent(basket_C_g)

print("=== Adults Male (%) ===")
display(top_A_m_pct)

print("=== Adults Female (%) ===")
display(top_A_f_pct)

print("=== Children Boys (%) ===")
display(top_C_b_pct)

print("=== Children Girls (%) ===")
display(top_C_g_pct)


=== Adults Male (%) ===


,Food,Percent_of_transactions
0,"Wheat bread and rolls, white (refined flour)",85.119714
1,"Salt, PROCESS= Raw, no heat treatment",49.339897
2,"Coffee (average strength) beverage, PROCESS= B...",44.394719
3,"Tap water, PROCESS= Boiling",42.157082
4,"Sunflower seed oil, edible, PROCESS= Raw, no h...",36.249720
5,"Sunflower seed oil, edible, PROCESS= Stewing, ...",34.302976
6,"Onions, PROCESS= Stewing, PROCESS= Pan frying ...",33.586932
7,"Carrots, PROCESS= Stewing, PROCESS= Pan frying...",26.784516
8,"Vegetables and vegetable products, GEN= Other,...",26.404117
9,"Tap water, PROCESS= Poaching",25.889461


=== Adults Female (%) ===


,Food,Percent_of_transactions
0,"Wheat bread and rolls, white (refined flour)",73.811815
1,"Coffee (average strength) beverage, PROCESS= B...",48.162401
2,"Salt, PROCESS= Raw, no heat treatment",48.066192
3,"Tap water, PROCESS= Boiling",39.349625
4,"Sunflower seed oil, edible, PROCESS= Raw, no h...",35.655186
5,"Sunflower seed oil, edible, PROCESS= Stewing, ...",32.249375
6,"Onions, PROCESS= Stewing, PROCESS= Pan frying ...",31.556667
7,"Vegetables and vegetable products, GEN= Other,...",27.977679
8,"Carrots, PROCESS= Stewing, PROCESS= Pan frying...",24.437175
9,"Tap water, PROCESS= Poaching",24.437175


=== Children Boys (%) ===


,Food,Percent_of_transactions
0,"Wheat bread and rolls, white (refined flour)",85.074627
1,"Salt, PROCESS= Raw, no heat treatment",49.004975
2,"Tap water, PROCESS= Boiling",42.537313
3,"Sunflower seed oil, edible, PROCESS= Stewing, ...",38.557214
4,"Onions, PROCESS= Stewing, PROCESS= Pan frying ...",37.313433
5,Milk,35.074627
6,"Sunflower seed oil, edible, PROCESS= Raw, no h...",31.840796
7,"Carrots, PROCESS= Stewing, PROCESS= Pan frying...",29.850746
8,"Salt, PROCESS= Stewing, PROCESS= Pan frying / ...",27.860697
9,"Herbal and other non-tea infusions, PROCESS= B...",27.363184


=== Children Girls (%) ===


,Food,Percent_of_transactions
0,"Wheat bread and rolls, white (refined flour)",78.167116
1,"Salt, PROCESS= Raw, no heat treatment",44.474394
2,"Tap water, PROCESS= Boiling",42.318059
3,"Sunflower seed oil, edible, PROCESS= Stewing, ...",33.962264
4,"Sunflower seed oil, edible, PROCESS= Raw, no h...",33.423181
5,"Onions, PROCESS= Stewing, PROCESS= Pan frying ...",33.153639
6,Milk,31.805930
7,Apples,28.032345
8,"Carrots, PROCESS= Stewing, PROCESS= Pan frying...",26.415094
9,"Herbal and other non-tea infusions, PROCESS= B...",26.415094


In [17]:
pct_children = (basket_C.sum() / basket_C.shape[0])
pct_adults = (basket_A.sum() / basket_A.shape[0])

diff = (pct_children - pct_adults).sort_values(ascending=False)

top_child_specific = (
    diff.head(15)
        .reset_index()
        .rename(columns={"index": "Food", 0: "Children_minus_Adults"})
)

display(top_child_specific)


,Food,Children_minus_Adults
0,Milk,0.178116
1,Tap water,0.136234
2,"Soft drink, flavoured, no fruit, PACKMAT= Plastic",0.110182
3,Breakfast cereals,0.077679
4,Common banana,0.076658
5,"Jam of fruit / vegetables, QUAL= With added su...",0.072617
6,"Pate, pork liver, STATE= Paste, PACKFORMAT= Th...",0.067589
7,"Meat soup, clear, PROCESS= Stewing, PROCESS= P...",0.060044
8,"Ice cream, milk-based",0.058332
9,Chocolate and chocolate products,0.058178


In [18]:
def top_items_percent_filtered(basket, drop_items, topN=15):
    b = basket.drop(columns=[c for c in drop_items if c in basket.columns], errors="ignore")
    pct = (b.sum() / b.shape[0]) * 100
    return (pct.sort_values(ascending=False)
              .head(topN)
              .reset_index()
              .rename(columns={"index":"Food", 0:"Percent_of_transactions"}))

# itemi “universali” = cei care apar în foarte multe tranzacții la ambele
common = ( (basket_A.sum()/basket_A.shape[0] > 0.45) & (basket_C.sum()/basket_C.shape[0] > 0.45) )
common_items = common[common].index.tolist()

print("Common items removed:", len(common_items))

print("=== Children top after removing common staples ===")
display(top_items_percent_filtered(basket_C, common_items, topN=15))

print("=== Adults top after removing common staples ===")
display(top_items_percent_filtered(basket_A, common_items, topN=15))


Common items removed: 2
=== Children top after removing common staples ===


,Food,Percent_of_transactions
0,"Tap water, PROCESS= Boiling",42.432083
1,"Sunflower seed oil, edible, PROCESS= Stewing, ...",36.351876
2,"Onions, PROCESS= Stewing, PROCESS= Pan frying ...",35.316947
3,Milk,33.505821
4,"Sunflower seed oil, edible, PROCESS= Raw, no h...",32.600259
5,"Carrots, PROCESS= Stewing, PROCESS= Pan frying...",28.201811
6,"Herbal and other non-tea infusions, PROCESS= B...",26.908150
7,Apples,26.908150
8,"Salt, PROCESS= Stewing, PROCESS= Pan frying / ...",26.778784
9,"Vegetables and vegetable products, GEN= Other,...",25.226391


=== Adults top after removing common staples ===


,Food,Percent_of_transactions
0,"Coffee (average strength) beverage, PROCESS= B...",46.420443
1,"Tap water, PROCESS= Boiling",40.647631
2,"Sunflower seed oil, edible, PROCESS= Raw, no h...",35.930064
3,"Sunflower seed oil, edible, PROCESS= Stewing, ...",33.198841
4,"Onions, PROCESS= Stewing, PROCESS= Pan frying ...",32.495345
5,"Vegetables and vegetable products, GEN= Other,...",27.250155
6,"Carrots, PROCESS= Stewing, PROCESS= Pan frying...",25.522450
7,"Tap water, PROCESS= Poaching",25.108628
8,"Salt, PROCESS= Stewing, PROCESS= Pan frying / ...",24.032692
9,Tomatoes and similar-,23.732671


In [19]:
def rules_table(rules_df, n=6):
    t = rules_df.head(n).copy()
    t["antecedents"] = t["antecedents"].apply(lambda s: ", ".join(list(s)))
    t["consequents"] = t["consequents"].apply(lambda s: ", ".join(list(s)))
    return t[["antecedents","consequents","support","confidence","lift"]]

def save_csv(df, filename):
    df.to_csv(filename, index=False)
    print("Saved:", filename)


In [24]:
table_adults_rules = rules_table(rules_A, n=6)
display(table_adults_rules)
save_csv(table_adults_rules, "tabel_6_1_reguli_adulti_top6.csv")


,antecedents,consequents,support,confidence,lift
1554,"Mammals and birds meat, PROCESS= Pan frying / ...","Garlic, PROCESS= Pan frying / shallow frying",0.033106,1.000000,28.597633
1553,"Garlic, PROCESS= Pan frying / shallow frying","Mammals and birds meat, PROCESS= Pan frying / ...",0.033106,0.946746,28.597633
1555,"Mammals and birds meat, PROCESS= Pan frying / ...","Wheat bread and rolls, white (refined flour), ...",0.033106,1.000000,27.617143
1556,"Wheat bread and rolls, white (refined flour), ...","Mammals and birds meat, PROCESS= Pan frying / ...",0.033106,0.914286,27.617143
1549,"Wheat bread and rolls, white (refined flour), ...","Garlic, PROCESS= Pan frying / shallow frying",0.034037,0.940000,26.881775
1548,"Garlic, PROCESS= Pan frying / shallow frying","Wheat bread and rolls, white (refined flour), ...",0.034037,0.973373,26.881775


Saved: tabel_6_1_reguli_adulti_top6.csv


In [21]:
keywords = ["milk", "sugar", "cocoa", "chocolate"]

mask = rules_C["antecedents"].apply(lambda s: any(k in " ".join(list(s)).lower() for k in keywords)) | \
       rules_C["consequents"].apply(lambda s: any(k in " ".join(list(s)).lower() for k in keywords))

rules_C_kw = rules_C[mask].copy()

# dacă sunt puține, ia top 6 după lift din tot setul
if len(rules_C_kw) < 6:
    rules_C_kw = rules_C.copy()

table_children_rules = rules_table(rules_C_kw, n=6)
display(table_children_rules)
save_csv(table_children_rules, "tabel_6_2_reguli_copii_top6.csv")


,antecedents,consequents,support,confidence,lift
873,"Cocoa powder, PROCESS= Reheating","White sugar, PROCESS= Reheating",0.03881,1.0,25.766667
874,"White sugar, PROCESS= Reheating","Cocoa powder, PROCESS= Reheating",0.03881,1.0,25.766667
876,"Milk, PROCESS= Reheating","White sugar, PROCESS= Reheating",0.03881,1.0,25.766667
877,"White sugar, PROCESS= Reheating","Milk, PROCESS= Reheating",0.03881,1.0,25.766667
878,"Milk, PROCESS= Reheating","Cocoa powder, PROCESS= Reheating",0.03881,1.0,25.766667
879,"Cocoa powder, PROCESS= Reheating","Milk, PROCESS= Reheating",0.03881,1.0,25.766667


Saved: tabel_6_2_reguli_copii_top6.csv


In [22]:
pct_children = (basket_C.sum() / basket_C.shape[0])
pct_adults   = (basket_A.sum() / basket_A.shape[0])

diff = (pct_children - pct_adults).sort_values(ascending=False)

table_child_minus_adult = (
    diff.head(15)
        .reset_index()
        .rename(columns={"index":"Food", 0:"Children_minus_Adults"})
)

display(table_child_minus_adult)
save_csv(table_child_minus_adult, "tabel_6_3_children_minus_adults_top15.csv")


,Food,Children_minus_Adults
0,Milk,0.178116
1,Tap water,0.136234
2,"Soft drink, flavoured, no fruit, PACKMAT= Plastic",0.110182
3,Breakfast cereals,0.077679
4,Common banana,0.076658
5,"Jam of fruit / vegetables, QUAL= With added su...",0.072617
6,"Pate, pork liver, STATE= Paste, PACKFORMAT= Th...",0.067589
7,"Meat soup, clear, PROCESS= Stewing, PROCESS= P...",0.060044
8,"Ice cream, milk-based",0.058332
9,Chocolate and chocolate products,0.058178


Saved: tabel_6_3_children_minus_adults_top15.csv


In [23]:
def top_items_table(basket, topN=5):
    return (
        basket.sum().sort_values(ascending=False).head(topN)
              .reset_index()
              .rename(columns={"index":"Food", 0:"Frequency"})
    )

t_A_m = top_items_table(basket_A_m, 5); t_A_m.insert(0, "Group", "Adults Male")
t_A_f = top_items_table(basket_A_f, 5); t_A_f.insert(0, "Group", "Adults Female")
t_C_b = top_items_table(basket_C_b, 5); t_C_b.insert(0, "Group", "Children Boys")
t_C_g = top_items_table(basket_C_g, 5); t_C_g.insert(0, "Group", "Children Girls")

table_gender_small = pd.concat([t_A_m, t_A_f, t_C_b, t_C_g], ignore_index=True)
display(table_gender_small)
save_csv(table_gender_small, "tabel_6_4_top5_pe_gen.csv")


,Group,Food,Frequency
0,Adults Male,"Wheat bread and rolls, white (refined flour)",3804
1,Adults Male,"Salt, PROCESS= Raw, no heat treatment",2205
2,Adults Male,"Coffee (average strength) beverage, PROCESS= B...",1984
3,Adults Male,"Tap water, PROCESS= Boiling",1884
4,Adults Male,"Sunflower seed oil, edible, PROCESS= Raw, no h...",1620
5,Adults Female,"Wheat bread and rolls, white (refined flour)",3836
6,Adults Female,"Coffee (average strength) beverage, PROCESS= B...",2503
7,Adults Female,"Salt, PROCESS= Raw, no heat treatment",2498
8,Adults Female,"Tap water, PROCESS= Boiling",2045
9,Adults Female,"Sunflower seed oil, edible, PROCESS= Raw, no h...",1853


Saved: tabel_6_4_top5_pe_gen.csv
